In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.base as meb
import numpy as np

### 网格数据插值到站点—临近点插值
**interp_gs_nearest(grd,sta,used_coords = "xy")**  
将水平网格数据插值到平面的离散站点上，对于每一个站点，其取值设置为周围四个网格点中最近的一个格点的取值。    

**参数说明：**  
 **grd**: 网格数据   
 **sta**: 站点数据         
 **used_coords**: 插值操作使用的维度，缺省情况下插值操作只在水平方向实现，此时返回的站点数据中id、lon、lat三列的取值会和sta一致，经纬度超出网格范围的站点将被删除，而level,time,dtime参数采用grd里的坐标信息。目前，该函数仅支持该参数缺省时的功能，该参数为其它选项时对应的功能待完善中。  
 **return**: 站点数据       


**调用示例**


In [2]:
grid0 = meb.grid([100,102,1],[20,22,1],gtime=["2019013108"],dtime_list = [24],level_list = [500,700],member_list = ["GRAPES"])
x= np.arange(3)
y= np.arange(3)
dat = np.array(np.meshgrid(x,y))
grd = meb.grid_data(grid0,dat)   #根据网格信息和numpy数组生成网格数
print(grd)

<xarray.DataArray 'data0' (member: 1, level: 2, time: 1, dtime: 1, lat: 3, lon: 3)>
array([[[[[[0, 1, 2],
           [0, 1, 2],
           [0, 1, 2]]]],



        [[[[0, 0, 0],
           [1, 1, 1],
           [2, 2, 2]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 500 700
  * time     (time) datetime64[ns] 2019-01-31T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) int32 20 21 22
  * lon      (lon) int32 100 101 102


In [3]:
station = meb.read_station(meb.station_国家站)  
meb.set_stadata_coords(station,level = 600)
print(station)

      level                time  dtime     id     lon    lat  data0
0       600 2099-01-01 08:00:00      0  50136  122.52  52.97      0
1       600 2099-01-01 08:00:00      0  50137  122.37  53.47      0
2       600 2099-01-01 08:00:00      0  50246  124.72  52.35      0
3       600 2099-01-01 08:00:00      0  50247  123.57  52.03      0
4       600 2099-01-01 08:00:00      0  50349  124.40  51.67      0
...     ...                 ...    ...    ...     ...    ...    ...
2406    600 2099-01-01 08:00:00      0  59945  109.70  18.65      0
2407    600 2099-01-01 08:00:00      0  59948  109.58  18.22      0
2408    600 2099-01-01 08:00:00      0  59951  110.33  18.80      0
2409    600 2099-01-01 08:00:00      0  59954  110.03  18.55      0
2410    600 2099-01-01 08:00:00      0  59981  112.33  16.83      0

[2411 rows x 7 columns]


In [4]:
sta = meb.interp_gs_nearest(grd,station)  
print(sta) #used_coords缺省时，返回数据的level,time和dtime和grd里取值一致,由于level有两层，因此插值结果中每个id会出现两次  

   level                time  dtime     id     lon    lat  GRAPES
0    500 2019-01-31 08:00:00     24  56958  100.42  21.92       0
1    500 2019-01-31 08:00:00     24  56959  100.78  22.00       0
2    500 2019-01-31 08:00:00     24  56969  101.58  21.47       1
3    700 2019-01-31 08:00:00     24  56958  100.42  21.92       1
4    700 2019-01-31 08:00:00     24  56959  100.78  22.00       2
5    700 2019-01-31 08:00:00     24  56969  101.58  21.47       1


### 网格数据插值到站点—双线性插值
**interp_gs_linear(grd,sta,used_coords ="xy")**  
将水平网格数据插值到平面的离散站点上，对于每一个站点，其取值利用周围四个网格点进行双线性插值获得      

**参数说明：**  
 **grd**: 网格数据   
 **sta**: 站点数据         
 **used_coords**: 插值操作使用的维度，该参数包括如下选项：   
    当该参数为"xy"时，插值操作只在水平方向实现，此时返回的站点数据中id、lon、lat三列的取值会和sta一致，经纬度超出网格范围的站点将被删除，而level,time,dtime参数采用grd里的坐标信息；   
    当该参数为"xyz"时，插值操作只在lon,lat,level三个空间维度实现，此时返回的站点数据中id、lon、lat,level四列的取值会和sta一致，经纬度超出网格范围的站点将被删除，而time,dtime参数采用grd里的坐标信息；
 **return**: 站点数据      


**调用示例**

In [5]:
sta = meb.interp_gs_linear(grd,station)   #在二维平面进行插值，level维度会
print(sta) #used_coords缺省时，返回数据的level,time和dtime和grd里取值一致,由于level有两层，因此插值结果中每个id会出现两次  

   level                time  dtime     id     lon    lat  GRAPES
0    500 2019-01-31 08:00:00     24  56958  100.42  21.92    0.42
1    500 2019-01-31 08:00:00     24  56959  100.78  22.00    0.78
2    500 2019-01-31 08:00:00     24  56969  101.58  21.47    1.58
3    700 2019-01-31 08:00:00     24  56958  100.42  21.92    1.92
4    700 2019-01-31 08:00:00     24  56959  100.78  22.00    2.00
5    700 2019-01-31 08:00:00     24  56969  101.58  21.47    1.47


In [6]:
sta = meb.interp_gs_linear(grd,station,used_coords = "xyz")
print(sta) #used_coords缺省时，返回数据的level,time和dtime和grd里取值一致,由于level有两层，因此插值结果中每个id会出现两次  

      level                time  dtime     id     lon    lat  GRAPES
1271    600 2019-01-31 08:00:00     24  56958  100.42  21.92   1.170
1272    600 2019-01-31 08:00:00     24  56959  100.78  22.00   1.390
1277    600 2019-01-31 08:00:00     24  56969  101.58  21.47   1.525


In [7]:
sta = meb.interp_gs_linear(grd,station,used_coords = "xydt")
print(sta) #used_coords缺省时，返回数据的level,time和dtime和grd里取值一致,由于level有两层，因此插值结果中每个id会出现两次 

dtime维度size = 1,无法开展dtime维度插值
None


### 网格数据插值到站点—双三次插值
**interp_gs_cubic(grd,sta,used_coords = "xy")**  
将水平网格数据插值到平面的离散站点上，对于每一个站点，其取值利用周围九个网格点进行双三次插值获得      

**参数说明：**  
 **grd**: 网格数据   
 **sta**: 站点数据         
 **used_coords**: 插值操作使用的维度，缺省情况下插值操作只在水平方向实现，此时返回的站点数据中id、lon、lat三列的取值会和sta一致，经纬度超出网格范围的站点将被删除，而level,time,dtime参数采用grd里的坐标信息。目前，该函数仅支持该参数缺省时的功能，该参数为其它选项时对应的功能待完善中。  
 **return**: 站点数据      

**调用示例**

In [8]:
sta =meb.interp_gs_cubic(grd,station)
print(sta)

   level                time  dtime     id     lon    lat    GRAPES
0    500 2019-01-31 08:00:00     24  56958  100.42  21.92  0.355852
1    500 2019-01-31 08:00:00     24  56959  100.78  22.00  0.745108
2    500 2019-01-31 08:00:00     24  56969  101.58  21.47  1.644148
3    700 2019-01-31 08:00:00     24  56958  100.42  21.92  1.943552
4    700 2019-01-31 08:00:00     24  56959  100.78  22.00  2.000000
5    700 2019-01-31 08:00:00     24  56969  101.58  21.47  1.531029


### 网格数据插值到网格—双线性插值
**interp_gg_linear(grd,grid,used_coords = "xy",outer_value = None)**  
将水平网格数据插值到平面的离散站点上，对于每一个站点，其取值利用周围四个网格点进行双线性插值获得      

**参数说明：**  
 **grd**: 网格数据(水平方向size不为1，其它维度size=1)   
 **grid**: 网格信息类变量，插值的目标网格。       
 **used_coords**: 插值操作使用的维度，缺省情况下插值操作只在水平方向实现，此时返回的网格数据中lon、lat维度的范围和间距由grid确定，其它维度信息由grd确定   
 **outer_value**: 当目标网格范围大于原始数据网格时，设置超出部分的取值为outer_value。如果存在网格超出的情况，就必须设置该参数，否则返回None。  
 **return**: 网格数据，网格范围由grid1确定          


**调用示例**

In [9]:
grid1 = meb.grid([100,102,0.5],[20,22,0.5])
grd1 = meb.interp_gg_linear(grd,grid1)
print(grd1)

<xarray.DataArray 'data0' (member: 1, level: 2, time: 1, dtime: 1, lat: 5, lon: 5)>
array([[[[[[0. , 0.5, 1. , 1.5, 2. ],
           [0. , 0.5, 1. , 1.5, 2. ],
           [0. , 0.5, 1. , 1.5, 2. ],
           [0. , 0.5, 1. , 1.5, 2. ],
           [0. , 0.5, 1. , 1.5, 2. ]]]],



        [[[[0. , 0. , 0. , 0. , 0. ],
           [0.5, 0.5, 0.5, 0.5, 0.5],
           [1. , 1. , 1. , 1. , 1. ],
           [1.5, 1.5, 1.5, 1.5, 1.5],
           [2. , 2. , 2. , 2. , 2. ]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 500 700
  * time     (time) datetime64[ns] 2019-01-31T08:00:00
  * dtime    (dtime) int32 24
  * lat      (lat) float64 20.0 20.5 21.0 21.5 22.0
  * lon      (lon) float64 100.0 100.5 101.0 101.5 102.0


In [10]:
grid1 = meb.grid([100,103,0.5],[20,23,0.5])
grd1 = meb.interp_gg_linear(grd,grid1)
print(grd1)

当目标网格超出数据网格时，outer_value参数必须赋值
None


In [11]:
grid1 = meb.grid([100,103,0.5],[20,23,0.5])
grd1 = meb.interp_gg_linear(grd,grid1,outer_value = -1)
print(grd1)

<xarray.DataArray 'data0' (member: 1, level: 2, time: 1, dtime: 1, lat: 7, lon: 7)>
array([[[[[[ 0. ,  0.5,  1. ,  1.5,  2. , -1. , -1. ],
           [ 0. ,  0.5,  1. ,  1.5,  2. , -1. , -1. ],
           [ 0. ,  0.5,  1. ,  1.5,  2. , -1. , -1. ],
           [ 0. ,  0.5,  1. ,  1.5,  2. , -1. , -1. ],
           [ 0. ,  0.5,  1. ,  1.5,  2. , -1. , -1. ],
           [-1. , -1. , -1. , -1. , -1. , -1. , -1. ],
           [-1. , -1. , -1. , -1. , -1. , -1. , -1. ]]]],



        [[[[ 0. ,  0. ,  0. ,  0. ,  0. , -1. , -1. ],
           [ 0.5,  0.5,  0.5,  0.5,  0.5, -1. , -1. ],
           [ 1. ,  1. ,  1. ,  1. ,  1. , -1. , -1. ],
           [ 1.5,  1.5,  1.5,  1.5,  1.5, -1. , -1. ],
           [ 2. ,  2. ,  2. ,  2. ,  2. , -1. , -1. ],
           [-1. , -1. , -1. , -1. , -1. , -1. , -1. ],
           [-1. , -1. , -1. , -1. , -1. , -1. , -1. ]]]]]])
Coordinates:
  * member   (member) <U6 'GRAPES'
  * level    (level) int32 500 700
  * time     (time) datetime64[ns] 2019-01-31T08:00:

### 站点数据插值到网格—反距离权重插值
**interp_sg_idw(sta, grid, background=None, effectR=1000, nearNum=8):**
将站点数据插值到水平网格上      

**参数说明：**  
 **sta**: 站点数据   
 **grid**: 网格信息类变量，插值的目标网格。       
 **background**: 插值背景场，在远离站点的区域内将采用background的值，如果background为None，则取为0 
 **effectR**: 最大的插值半径  
 **nearNum**: 插值选择的临近站点的个数, nearNum 必须大于1    
 **return**: 网格数据，网格范围由grid1确定          


**调用示例**

In [12]:
grd2 = meb.interp_sg_idw(sta,grid1,nearNum = 2)
print(grd2)

<xarray.DataArray 'data0' (member: 1, level: 1, time: 1, dtime: 1, lat: 7, lon: 7)>
array([[[[[[1.1497020000000016, 1.5875887499999985, 1.5875887499999988,
            1.5875887499999985, 1.5875887499999988, 1.5875887499999988,
            1.5875887499999988],
           [1.1497020000000016, 1.5875887499999985, 1.5875887499999988,
            1.5875887499999985, 1.5875887499999985, 1.5875887499999985,
            1.5875887499999988],
           [1.1497020000000016, 1.1497020000000016, 1.587588749999999,
            1.5875887499999988, 1.5875887499999985, 1.5875887499999985,
            1.5875887499999988],
           [1.1497020000000016, 1.1497020000000016, 1.3725540000000007,
            1.5875887499999988, 1.5875887499999985, 1.5875887499999985,
            1.5875887499999985],
           [1.1497020000000016, 1.1497020000000016, 1.3725540000000007,
            1.5875887499999985, 1.5875887499999988, 1.5875887499999985,
            1.5875887499999985],
           [1.1497020000000016, 